In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline

In [2]:
import lightgbm as lgb

In [3]:
data_path = './data/jlr_all_user_processed.xlsx'
df = pd.read_excel(data_path)
df.head()

,user_id,follow_count,fans_count,gender,birthday,location,level,registration_time,properties,mileage,...,label,car_like,excellent_post_count,all_post_count,regis_year,regis_month,regis_day,regis_time_relative,date,date_delta
0,oden123,0,0,2,NaT,1,1,2018-03-26,1,4430,...,True,1,0,0,2018,3,3,6660,2018-03-26,4522
1,生活1934626,2,4,2,NaT,2,1,2012-05-07,1,4270,...,True,2,0,0,2012,5,5,4511,2012-05-07,2373
2,wangzi1125,0,1,2,NaT,3,1,2017-06-18,1,2090,...,True,3,0,1,2017,6,6,6379,2017-06-18,4241
3,房产专家谢广财,3,2,2,NaT,4,1,2014-02-27,1,1120,...,True,4,0,0,2014,2,2,5172,2014-02-27,3034
4,南宫晗笑,4,5,2,NaT,5,1,2017-07-03,1,740,...,True,1,0,14,2017,7,7,6394,2017-07-03,4256


In [4]:
df.keys()

Index(['user_id', 'follow_count', 'fans_count', 'gender', 'birthday',
       'location', 'level', 'registration_time', 'properties', 'mileage',
       'post', 'cars', 'koubei_post', 'gas_mileage', 'car_friend_zone',
       'label', 'car_like', 'excellent_post_count', 'all_post_count',
       'regis_year', 'regis_month', 'regis_day', 'regis_time_relative', 'date',
       'date_delta'],
      dtype='object')

In [5]:
df.dtypes

user_id                         object
follow_count                     int64
fans_count                       int64
gender                           int64
birthday                datetime64[ns]
location                         int64
level                            int64
registration_time       datetime64[ns]
properties                       int64
mileage                          int64
post                            object
cars                            object
koubei_post                     object
gas_mileage                     object
car_friend_zone                 object
label                             bool
car_like                         int64
excellent_post_count             int64
all_post_count                   int64
regis_year                       int64
regis_month                      int64
regis_day                        int64
regis_time_relative              int64
date                    datetime64[ns]
date_delta                       int64
dtype: object

In [6]:
x = df.drop(columns=[
                      'user_id',
#                      'follow_count',
#                      'fans_count', 
#                      'gender', 
                      'birthday',
#                      'location',
#                      'level',
                      'registration_time',
#                      'properties',
#                      'mileage',
                      'post',
                      'cars',
                      'koubei_post',
                      'gas_mileage',
                      'car_friend_zone',
                      'label',
#                      'car_like', 
#                      'excellent_post_count',
#                      'all_post_count',
                      'regis_year',
                      'regis_month',
                      'regis_day', 
                      'regis_time_relative',
                      'date',
#                      'date_delta'
                     ])
x.head(2)

,follow_count,fans_count,gender,location,level,properties,mileage,car_like,excellent_post_count,all_post_count,date_delta
0,0,0,2,1,1,1,4430,1,0,0,4522
1,2,4,2,2,1,1,4270,2,0,0,2373


In [7]:
y = df['label'].astype('uint8')
y.describe()

count    5555.000000
mean        0.130513
std         0.336897
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [37]:
_x, _y = np.array(x).astype('int'), np.array(y).astype('int')
_x.shape, _y.shape

((5555, 11), (5555,))

### Split dataset into train/validation/test set

In [38]:
x_train, x_test, y_train, y_test = train_test_split(_x, _y, test_size=0.2,  random_state = 42)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5,  random_state = 42)

In [39]:
#print(x_train.shape, y_train.shape)

#y_test.describe()

In [40]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,

### Build SVM model then train it

In [41]:
clf = SVC(C=0.8, kernel='rbf', class_weight='balanced')

In [42]:
clf.fit(x_train, y_train)

SVC(C=0.8, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [43]:
pred = clf.predict(x_test)

In [44]:
print(classification_report(y_true=y_test, y_pred=pred))

             precision    recall  f1-score   support

          0       0.85      0.92      0.88       476
          1       0.03      0.01      0.02        80

avg / total       0.73      0.79      0.76       556



### Build lightgbm model and train it 

#### Dataset construction based on numpy data

In [54]:
x.keys(), x.dtypes

(Index(['follow_count', 'fans_count', 'gender', 'location', 'level',
        'properties', 'mileage', 'car_like', 'excellent_post_count',
        'all_post_count', 'date_delta'],
       dtype='object'), follow_count            int64
 fans_count              int64
 gender                  int64
 location                int64
 level                   int64
 properties              int64
 mileage                 int64
 car_like                int64
 excellent_post_count    int64
 all_post_count          int64
 date_delta              int64
 dtype: object)

In [46]:
x_train[:2], y_train[:2]

(array([[  11,   12,    2,   14,    1,    1, 1330,  117,    0,    4, 3404],
        [  16,   24,    2,   15,    1,    1, 7310,    1,    1,   30, 1969]]),
 array([0, 0]))

In [47]:
train_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_val, label=y_val)
test_data = lgb.Dataset(x_test, label = y_test)

In [55]:
feature_name = list(x.keys())
categorical_feature = ['gender','location', 'level', 
                     'properties', 'car_like',
                     #'regis_year', 'regis_month', 'regis_day',
                                            ]
train_data = lgb.Dataset(x_train, 
                         label=y_train,
                         feature_name=feature_name,
                         categorical_feature=categorical_feature,
                        )
val_data = lgb.Dataset(x_val, 
                       label=y_val,
                       feature_name=feature_name,
                       categorical_feature=categorical_feature,
                      )
test_data = lgb.Dataset(x_test, 
                        label = y_test,
                        feature_name=feature_name,
                        categorical_feature=categorical_feature,
                       )


In [56]:
 feature_name, categorical_feature

(['follow_count',
  'fans_count',
  'gender',
  'location',
  'level',
  'properties',
  'mileage',
  'car_like',
  'excellent_post_count',
  'all_post_count',
  'date_delta'],
 ['gender', 'location', 'level', 'properties', 'car_like'])

#### Setting parameters

In [57]:
# param = {'max_depth': 10,
#         'num_leaves': 20,
#         'num_trees': 5,
#         'objective':'binary',
#         'min_data_in_leaf':100}
# param['metric'] = 'auc'

# para searching
params = {
    'boosting_type': 'gbdt',  # np.random.choice(['dart', 'gbdt']),
    'objective': 'binary',
    'metric': ['binary_logloss', 'auc'], 
    
    'learning_rate': 0.35,
    
    'num_leaves': np.random.randint(23),
    'max_depth': np.random.randint(7,10),
    'min_data_in_leaf': int(2 ** (np.random.rand()*1.5 + 5)),
    
    'feature_fraction': np.random.rand()*0.35+0.65,
    'bagging_fraction': np.random.rand()*0.35+0.65,
    'bagging_freq': 1,
    
    'lambda_l1': 10 ** (np.random.rand() * 4),
    'lambda_l2': 10 ** (np.random.rand() * 3 + 2),
    'min_gain_to_split': 0.0,
    'min_sum_hessian_in_leaf': 0.1,
    
    'num_threads': 16,
    'verbose': 0,
    'is_training_metric': 'True'
}
print('Hyper-parameters:')
print(params)

Hyper-parameters:
{'boosting_type': 'gbdt', 'objective': 'binary', 'metric': ['binary_logloss', 'auc'], 'learning_rate': 0.35, 'num_leaves': 15, 'max_depth': 9, 'min_data_in_leaf': 47, 'feature_fraction': 0.7353692081590242, 'bagging_fraction': 0.9128047176996665, 'bagging_freq': 1, 'lambda_l1': 7681.519650788639, 'lambda_l2': 2209.845772592949, 'min_gain_to_split': 0.0, 'min_sum_hessian_in_leaf': 0.1, 'num_threads': 16, 'verbose': 0, 'is_training_metric': 'True'}


In [58]:
params={'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': ['binary_logloss', 'auc'],
        'learning_rate': 0.3,
        'num_leaves': 10,
        'max_depth': 9,
        'min_data_in_leaf': 100,
        'feature_fraction': 0.8815622868115178,
        'bagging_fraction': 0.8390135637666467, 
        'bagging_freq': 1, 
        'lambda_l1': 31.388095945215674,
        'lambda_l2': 9057.459915710713,
        'min_gain_to_split': 0.0,
        'min_sum_hessian_in_leaf': 0.1, 
        'num_threads': 16, 'verbose': 0,
        'is_training_metric': 'True'}

#### Training 

In [59]:
# num_round = 100
# bst = lgb.train(param, 
#                 train_data,
#                 num_round,
#                 valid_sets=[val_data],
#                 early_stopping_rounds=30
#                )

evals_result = {}  # dict to store evaluation results of all the items in `valid_sets`

gbm = lgb.train(params, 
                train_data, 
                5000, 
                valid_sets=[train_data, val_data], \
                valid_names = ['train', 'valid'], 
                evals_result=evals_result, \
                early_stopping_rounds=3000, 
                verbose_eval=200)

/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1040: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:685: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 3000 rounds.
[200]	train's auc: 0.796604	train's binary_logloss: 0.343769	valid's auc: 0.711647	valid's binary_logloss: 0.339752
[400]	train's auc: 0.805772	train's binary_logloss: 0.32698	valid's auc: 0.718268	valid's binary_logloss: 0.33036
[600]	train's auc: 0.810707	train's binary_logloss: 0.317607	valid's auc: 0.725792	valid's binary_logloss: 0.325249
[800]	train's auc: 0.81516	train's binary_logloss: 0.311243	valid's auc: 0.729661	valid's binary_logloss: 0.321641
[1000]	train's auc: 0.819402	train's binary_logloss: 0.306551	valid's auc: 0.731588	valid's binary_logloss: 0.319293
[1200]	train's auc: 0.822459	train's binary_logloss: 0.303114	valid's auc: 0.732506	valid's binary_logloss: 0.317584
[1400]	train's auc: 0.825437	train's binary_logloss: 0.300316	valid's auc: 0.735105	valid's binary_logloss: 0.316218
[1600]	train's auc: 0.827669	train's binary_logloss: 0.298256	valid's auc: 0.73639	valid's binary_logloss: 0.315257
[1800]	t

In [60]:
# save best round results to csv
bst_round = np.argmax(evals_result['valid']['auc'])
trn_auc = evals_result['train']['auc'][bst_round]
trn_loss = evals_result['train']['binary_logloss'][bst_round]
val_auc = evals_result['valid']['auc'][bst_round]
val_loss = evals_result['valid']['binary_logloss'][bst_round]

In [61]:
print('Best Round: %d'%bst_round)
print('Training loss: %.5f, Validation loss: %.5f'%(trn_loss, val_loss))
print('Training AUC : %.5f, Validation AUC : %.5f'%(trn_auc, val_auc))

Best Round: 2190
Training loss: 0.29448, Validation loss: 0.31326
Training AUC : 0.83146, Validation AUC : 0.73933


In [62]:
feature_importance = pd.DataFrame({'name':gbm.feature_name(), 'importance':gbm.feature_importance()}).sort_values(by='importance', ascending=False)
feature_importance.to_csv('./feat_importance.csv', index=False)

In [63]:
#  save model
# json_model = bst.dump_model()

#### Prediction

In [64]:
y_pred = gbm.predict(x_test)
print(y_pred[:10])

y_pred = y_pred >= 0.5
print(y_pred[:10])

from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(y_test, y_pred))
print(classification_report(y_true=y_test, y_pred=y_pred))

[0.06488052 0.07806898 0.03929476 0.04572762 0.0820784  0.04195991
 0.2280966  0.44026437 0.54285576 0.0691763 ]
[False False False False False False False False  True False]
0.8866906474820144
             precision    recall  f1-score   support

          0       0.89      0.99      0.94       476
          1       0.79      0.29      0.42        80

avg / total       0.88      0.89      0.86       556

